# Task_002 - Criar Volumes e Carregar Dados no Schema 00_landing

Este notebook cria volumes no schema `00_landing` e carrega os arquivos CSV da pasta `data` para dentro dos volumes.

## Arquivos a serem carregados:
- `claims.csv` → Volume para dados de sinistros
- `customers.csv` → Volume para dados de clientes
- `policies.csv` → Volume para dados de apólices

## O que são Volumes no Unity Catalog?
- **Volumes** são containers de arquivos no Unity Catalog
- Permitem armazenar arquivos não estruturados ou semi-estruturados (CSV, JSON, Parquet, etc.)
- Fornecem governança e controle de acesso granular
- São a alternativa moderna ao DBFS para armazenamento de arquivos



## Parte 1: Garantir que o Catálogo e Schema existem

Primeiro, vamos garantir que estamos usando o catálogo correto e que o schema 00_landing existe.



In [ ]:
-- Usar o catálogo smart_claims_dev
USE CATALOG smart_claims_dev;

-- Verificar se o schema 00_landing existe
SHOW SCHEMAS IN CATALOG smart_claims_dev LIKE '00_landing';



## Parte 2: Criar Volumes no Schema 00_landing

**COMANDO: CREATE VOLUME**

O que faz:
- Cria um volume dentro de um schema para armazenar arquivos
- Volumes são containers para arquivos não estruturados ou semi-estruturados
- Permitem controle de acesso granular através do Unity Catalog
- Suportam diferentes tipos de arquivo (CSV, JSON, Parquet, etc.)



In [ ]:
-- Criar volume para dados de sinistros (claims)
CREATE VOLUME IF NOT EXISTS smart_claims_dev.`00_landing`.claims_volume
COMMENT 'Volume para armazenar arquivos CSV de sinistros';



In [ ]:
-- Criar volume para dados de clientes (customers)
CREATE VOLUME IF NOT EXISTS smart_claims_dev.`00_landing`.customers_volume
COMMENT 'Volume para armazenar arquivos CSV de clientes';



In [ ]:
-- Criar volume para dados de apólices (policies)
CREATE VOLUME IF NOT EXISTS smart_claims_dev.`00_landing`.policies_volume
COMMENT 'Volume para armazenar arquivos CSV de apólices';



## Parte 3: Verificar Volumes Criados



In [ ]:
-- Listar todos os volumes no schema 00_landing
SHOW VOLUMES IN smart_claims_dev.`00_landing`;



## Parte 4: Carregar Arquivos CSV nos Volumes

### Método 1: Usando Python para Upload de Arquivos

Para fazer upload dos arquivos CSV para os volumes, você tem duas opções:

1. **Upload via UI do Databricks**: Arraste e solte os arquivos na interface
2. **Upload via código Python**: Use o código abaixo em uma célula Python



In [ ]:
# ============================================================
# UPLOAD DE ARQUIVOS CSV PARA OS VOLUMES USANDO DBUTILS
# Execute esta célula Python para fazer upload dos arquivos
# ============================================================

# IMPORTANTE: Ajuste os caminhos abaixo para corresponder ao seu ambiente
# Opção 1: Se os arquivos estão em um repositório Git conectado ao Databricks
repo_path = "/Workspace/Repos/your_username/smart_claims_dev/data"  # Ajuste conforme necessário

# Opção 2: Se você fez upload manual para DBFS primeiro
dbfs_path = "dbfs:/FileStore/uploads"  # Ajuste conforme necessário

# Caminhos dos volumes no Unity Catalog
volume_paths = {
    'claims': '/Volumes/smart_claims_dev/00_landing/claims_volume/claims.csv',
    'customers': '/Volumes/smart_claims_dev/00_landing/customers_volume/customers.csv',
    'policies': '/Volumes/smart_claims_dev/00_landing/policies_volume/policies.csv'
}

# Método 1: Se os arquivos estão em um repositório ou caminho local acessível
try:
    # Copiar de repositório para volumes
    dbutils.fs.cp(f"{repo_path}/claims.csv", volume_paths['claims'])
    dbutils.fs.cp(f"{repo_path}/customers.csv", volume_paths['customers'])
    dbutils.fs.cp(f"{repo_path}/policies.csv", volume_paths['policies'])
    print("✅ Arquivos copiados com sucesso do repositório para os volumes!")
except Exception as e:
    print(f"⚠️ Erro ao copiar do repositório: {str(e)}")
    print("   Tentando método alternativo via DBFS...")
    
    # Método 2: Se os arquivos estão no DBFS
    try:
        dbutils.fs.cp(f"{dbfs_path}/claims.csv", volume_paths['claims'])
        dbutils.fs.cp(f"{dbfs_path}/customers.csv", volume_paths['customers'])
        dbutils.fs.cp(f"{dbfs_path}/policies.csv", volume_paths['policies'])
        print("✅ Arquivos copiados com sucesso do DBFS para os volumes!")
    except Exception as e2:
        print(f"⚠️ Erro ao copiar do DBFS: {str(e2)}")
        print("\n📝 Alternativas:")
        print("   1. Faça upload manual via UI do Databricks (recomendado)")
        print("   2. Ajuste os caminhos acima para corresponder ao seu ambiente")
        print("   3. Use dbutils.fs.put() para fazer upload direto via código")



In [ ]:
# ============================================================
# UPLOAD VIA DBFS E CÓPIA PARA VOLUMES
# ============================================================

# Passo 1: Fazer upload dos arquivos para DBFS (usar UI ou dbutils)
# dbutils.fs.put("dbfs:/FileStore/uploads/claims.csv", "conteúdo_do_arquivo")

# Passo 2: Copiar de DBFS para os volumes
# dbutils.fs.cp("dbfs:/FileStore/uploads/claims.csv", "/Volumes/smart_claims_dev/00_landing/claims_volume/claims.csv")
# dbutils.fs.cp("dbfs:/FileStore/uploads/customers.csv", "/Volumes/smart_claims_dev/00_landing/customers_volume/customers.csv")
# dbutils.fs.cp("dbfs:/FileStore/uploads/policies.csv", "/Volumes/smart_claims_dev/00_landing/policies_volume/policies.csv")

print("📝 Descomente e execute os comandos acima após fazer upload dos arquivos para DBFS")



## Parte 5: Verificar Arquivos nos Volumes

Após fazer upload dos arquivos, verifique se eles estão nos volumes:



In [ ]:
# Listar arquivos no volume de claims
print("=== Arquivos no volume claims_volume ===")
try:
    files = dbutils.fs.ls("/Volumes/smart_claims_dev/00_landing/claims_volume/")
    for file in files:
        print(f"  - {file.name} ({file.size} bytes)")
except Exception as e:
    print(f"  ⚠️ Erro ao listar: {str(e)}")

print("\n=== Arquivos no volume customers_volume ===")
try:
    files = dbutils.fs.ls("/Volumes/smart_claims_dev/00_landing/customers_volume/")
    for file in files:
        print(f"  - {file.name} ({file.size} bytes)")
except Exception as e:
    print(f"  ⚠️ Erro ao listar: {str(e)}")

print("\n=== Arquivos no volume policies_volume ===")
try:
    files = dbutils.fs.ls("/Volumes/smart_claims_dev/00_landing/policies_volume/")
    for file in files:
        print(f"  - {file.name} ({file.size} bytes)")
except Exception as e:
    print(f"  ⚠️ Erro ao listar: {str(e)}")



## Parte 6: Ler Arquivos dos Volumes como DataFrames (Opcional)

Após os arquivos estarem nos volumes, você pode ler diretamente como DataFrames Spark:



In [ ]:
# Ler arquivos CSV dos volumes como DataFrames Spark
# Descomente após fazer upload dos arquivos

# df_claims = spark.read \
#     .option("header", "true") \
#     .option("inferSchema", "true") \
#     .csv("/Volumes/smart_claims_dev/00_landing/claims_volume/claims.csv")

# df_customers = spark.read \
#     .option("header", "true") \
#     .option("inferSchema", "true") \
#     .csv("/Volumes/smart_claims_dev/00_landing/customers_volume/customers.csv")

# df_policies = spark.read \
#     .option("header", "true") \
#     .option("inferSchema", "true") \
#     .csv("/Volumes/smart_claims_dev/00_landing/policies_volume/policies.csv")

# Mostrar preview dos dados
# print("=== Preview: Claims ===")
# df_claims.show(5, truncate=False)
# print(f"\nTotal de registros: {df_claims.count()}")

# print("\n=== Preview: Customers ===")
# df_customers.show(5, truncate=False)
# print(f"\nTotal de registros: {df_customers.count()}")

# print("\n=== Preview: Policies ===")
# df_policies.show(5, truncate=False)
# print(f"\nTotal de registros: {df_policies.count()}")

print("📝 Descomente o código acima após fazer upload dos arquivos para os volumes")

